In [ ]:
!wget https://uc.hackerearth.com/he-public-data/dataset1c2f4b7.zip

--2024-09-23 13:10:47--  https://uc.hackerearth.com/he-public-data/dataset1c2f4b7.zip
Resolving uc.hackerearth.com (uc.hackerearth.com)... 13.35.210.75, 13.35.210.90, 13.35.210.84, ...
Connecting to uc.hackerearth.com (uc.hackerearth.com)|13.35.210.75|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 621018124 (592M) [binary/octet-stream]
Saving to: ‘dataset1c2f4b7.zip’

dataset1c2f4b7.zip  100%[===================>] 592.25M   381MB/s    in 1.6s    

2024-09-23 13:10:48 (381 MB/s) - ‘dataset1c2f4b7.zip’ saved [621018124/621018124]



In [ ]:
!unzip dataset1c2f4b7.zip

Archive:  dataset1c2f4b7.zip
   creating: dataset/
  inflating: dataset/sample_submission.csv  
  inflating: dataset/test.csv        
  inflating: dataset/train.csv       


In [ ]:
import numpy as np
import pandas as pd
import json
df = pd.read_csv('/content/dataset/train.csv',encoding='latin-1')

In [ ]:
import re
import unicodedata
import numpy as np

def clean_text(text):
    # Normalize unicode characters
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    text = re.sub(r'[\x00-\x1f\x7f-\x9f]', '', text)
    text = re.sub(r'•|\u2022|\u25E6|\u2023|\u25CF', '-', text)  # Replace various bullet characters with '-'
    text = re.sub(r'\*+', '', text)  # Remove asterisks used for obfuscation
    text = re.sub(r'\s+', ' ', text)  # Turn all whitespace into single spaces, including newlines
    text = re.sub(r'\s*([()])\s*', r'\1', text)
    return text.strip()

# df['text'] = df['text'].apply(clean_text)

In [ ]:
df.head()

,text,target,Word Count
0,"python courses python courses, python exercise...",academic interests,125
1,the learning point open digital education. a r...,academic interests,147
2,"tech news, latest technology, mobiles, laptops...",academic interests,143
3,the best it certification materials in usa | k...,academic interests,364
4,"bioland scientific, for your research needs bi...",academic interests,176


In [ ]:
import torch
class DialogueTemplate():
    # system_token = "<|system|>" #system prompt
    user_token = "<|user|>"
    assistant_token = "<|assistant|>"
    # end_token = "<|end|>"

    messages = []

    def get_special_tokens(self):
        return [self.user_token, self.assistant_token]

    def get_training_prompt(self, infer = False):
        prompt = self.user_token + clean_text(self.messages['text']) + '\n' + self.assistant_token + self.messages['target']
        if infer == False:
          prompt += '<|endoftext|>'
        # prompt = prompt +  ' <|endoftext|>'
        return prompt #fully formed tranining prompt

    def get_inference_prompt(self, text_c):
        prompt = self.user_token + text_c + '\n' + self.assistant_token
        return prompt

    def get_raw_dialogue(self):
        prompt = ''
        for message in self.messages:
            prompt += message["content"] + '\n'

        return prompt

    def prepare_dialogue(self, example):
        self.messages = example# for rpg quest
        # print("Message is:", self.messages)

        # example['text'] = self.get_training_prompt()
        return self.get_training_prompt()

def get_dialogue_template():
    return DialogueTemplate()



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from pickle import FALSE
import torch
from torch.utils.data import Dataset
from random import shuffle

# from dialogue import get_dialogue_template

class  DialogueDataset(Dataset):
    def __init__(self, data, tokenizer, max_samples, seq_length=1024):
        super().__init__()

        self.data = data
        self.tokenizer = tokenizer
        self.seq_length = seq_length
        self.dialogue_template = get_dialogue_template()

        self.shuffle()
        print("DATA LENGTH: ", len(data))
        if max_samples > 0:
            self.data = self.data[:max_samples]

        # dialogue_tokens = self.dialogue_template.get_special_tokens()
        # print("dialogue tokens are: ",dialogue_tokens)
        # self.tokenizer.add_special_tokens({"additional_special_tokens": dialogue_tokens})
        # self.tokenizer.add_special_tokens({'pad_token':'<pad>'})

    def shuffle(self):
        self.data = self.data.sample(frac=1).reset_index(drop=True)
        return self.data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data.iloc[idx]
        diag = self.dialogue_template.prepare_dialogue(sample)

        input = self.tokenizer(diag, return_token_type_ids=False,max_length=self.seq_length,truncation=True,padding='max_length',return_tensors='pt')

        input['input_ids'] = input['input_ids'].squeeze(0)
        input['attention_mask'] = input['attention_mask'].squeeze(0)
        # labels = torch.empty_like(input['input_ids']).copy_(input["input_ids"])
        labels = torch.empty_like(input['input_ids']).copy_(input["input_ids"])

        labels = self.mask_user_labels(labels)

        # labels = torch.cat([labels[1:], torch.full((1,), self.tokenizer.encode('<pad>')[0],dtype=labels.dtype)],dim=0)

        return {
            "input_ids" : input["input_ids"],
            "attention_mask" :input["attention_mask"],
            "labels":labels,
        }
    def mask_user_labels(self, labels):
        user_token_id = self.tokenizer.convert_tokens_to_ids(self.dialogue_template.user_token)
        assistant_token_id = self.tokenizer.convert_tokens_to_ids(self.dialogue_template.assistant_token)
        # system_token_id = self.tokenizer.convert_tokens_to_ids(self.dialogue_template.system_token)
        pad_token_id = self.tokenizer.convert_tokens_to_ids('<|pad|>')
        masking = False
        for idx, label_id in enumerate(labels):
            marked = False
            if label_id in [user_token_id]:
                masking = True  # Start masking when user token is encountered
            if label_id == assistant_token_id:
                labels[idx] = pad_token_id  # Ensure assistant token is masked
                masking = False
            if masking:
                labels[idx] = pad_token_id  # Mask current token if in masking mode

        return labels


def get_dialogue_dataset(data, tokenizer, max_samples,seq_length):
    return DialogueDataset(data, tokenizer, max_samples, seq_length)

In [ ]:
import os
import json
import torch
from torch.utils.data import DataLoader, random_split
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR
from transformers import AutoModelForCausalLM, AutoTokenizer, get_linear_schedule_with_warmup
from tqdm import tqdm

def load_from_file(filename):
    with open(filename, 'r', encoding='utf-8',errors="ignore") as f:
        return json.load(f)

def decode_masked_labels(labels, tokenizer):
    # Replace -100 with tokenizer.pad_token_id
    pad_id = tokenizer.pad_token_id if tokenizer.pad_token_id is not None else tokenizer.eos_token_id
    labels = torch.where(labels != -100, labels, torch.tensor(pad_id))
    # Decode the labels
    return tokenizer.decode(labels)

model_name = "distilgpt2"
train_batch_size = 16
test_batch_size = 4

def get_ds(model_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    # if tokenizer.pad_token is None:
    #     tokenizer.pad_token = tokenizer.eos_token
    dialogue_template = get_dialogue_template()
    special_tokens = dialogue_template.get_special_tokens()

    tokenizer.add_special_tokens({'additional_special_tokens': special_tokens})
    tokenizer.add_special_tokens({'pad_token': '<|pad|>'})
    # raw_dataset = load_from_file(data_filename)
    dataset = get_dialogue_dataset(df, tokenizer, 800000, 1024)


    val_size = int(0.1 * len(dataset))
    train_size = len(dataset) - val_size
    train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
    train_dataloader = DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True, pin_memory=False)
    val_dataloader = DataLoader(val_dataset, batch_size=test_batch_size, shuffle=True)

    return train_dataloader, val_dataloader, tokenizer

from collections import Counter
def analyze_invalid_tokens(dataloader, tokenizer):
    invalid_token_counter = Counter()
    total_tokens = 0

    for batch in tqdm(dataloader, desc="Analyzing invalid tokens"):
        labels = batch["labels"]
        invalid_ids = (labels < 0) | (labels >= len(tokenizer))
        invalid_token_counter.update(labels[invalid_ids].tolist())
        total_tokens += labels.numel()

    print(f"Total tokens: {total_tokens}")
    # print(f"Total invalid tokens: {sum(invalid_token_counter.values())}")
    print(f"Unique invalid tokens: {len(invalid_token_counter)}")
    print("Top 10 invalid tokens:")
    for token, count in invalid_token_counter.most_common(10):
        token_str = tokenizer.decode([token]) if token >= len(tokenizer) else ""
        print(f"Token ID {token}: {token_str} - {count} occurrences")

    return invalid_token_counter
# data_filename = r"G:\Coding\gpt2\datasets\internet_archive_scifi_v3.txt"
# data_filename = r"/content/gdrive/MyDrive/GPT2-Finetuning/finetune_chatbot/finetune_chatbot/data/oasst.json"
# data_filename = data_file
train_dataloader, val_dataloader, tokenizer = get_ds(model_name)
for batch in train_dataloader:
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]

    labels=batch["labels"]

    print("input_ids: ", tokenizer.decode(input_ids[0]))
    print("labels: ", decode_masked_labels(labels[0], tokenizer))
    print("attention_mask: ", attention_mask[0])

    break
# analyze_invalid_tokens(val_dataloader, tokenizer)

print("Special: " ,tokenizer.additional_special_tokens)
special_token_ids = tokenizer.convert_tokens_to_ids(tokenizer.additional_special_tokens)
print("vocab size: ", len(tokenizer.vocab), " ids: ", special_token_ids)
tokenized_size = []
lines = 0
sum =  0
for id,row in tqdm(df.iterrows(), total=len(df)):
    tokenized_input = tokenizer.tokenize(row['text'])
    tokenized_size.append(len(tokenized_input))
    lines += 1
    if lines > 500:
        break
# print(tokenized_size)
# np.histogram(tokenized_size)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


DATA LENGTH:  697527
input_ids:  <|user|>in the same corner. I sat on it for the first time just started crying saying, i12i12i12Ya Allah please forgive my sins.i12i12i12 That was 2012. I started reading the Quran and the translations and it connected with me really fast. I started practising Islam and learnt how to pray and by January 2014, felt sure about converting. Since I am known as Yuvan Shankar Raja in films, I have still not changed my name officially in my passport and other records, but maybe later, I might do that too. My father was the last one I told in my house to. I told him, i12i12i12I have started reading the Quran and it gives me a lot of peace.i12i12i12 He said to me, i12i12i12Yuvan, I am not comfortable with you becoming Islamic.i12i12i12 My brother and his wife were very supportive. Iti12i12i12s odd but in some way I used to get that spiritual feeling that it was my mom, who held my hand, and said, i12i12i12Yuvan, you are alone. I want you to stand here under the 

  0%|          | 500/697527 [00:00<17:38, 658.35it/s]


In [ ]:
import torch
def finetune_gpt2(train_dataloader, val_dataloader, tokenizer, model, epochs=2, learning_rate=5e-5, warmup_steps=1000, max_grad_norm=1.0, device="cuda"):
    # tokenizer = AutoTokenizer.from_pretrained(model_name)
    model.resize_token_embeddings(len(tokenizer))

    optimizer = AdamW(model.parameters(), lr=learning_rate)
    total_steps = len(train_dataloader) * epochs
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=total_steps)
    pad_token_id = tokenizer.pad_token_id

    for epoch in range(epochs):
        total_loss = 0
        model.train()
        for batch in tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{epochs}"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            labels[labels == pad_token_id] = -100

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss

            total_loss += loss.item()

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()

        avg_train_loss = total_loss / len(train_dataloader)
        print(f"Average train loss: {avg_train_loss:.4f}")

        # Validation
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for batch in tqdm(val_dataloader, desc="Validation"):
                input_ids = batch["input_ids"].to(device)
                attention_mask = batch["attention_mask"].to(device)
                labels = batch["labels"].to(device)

                labels[labels == pad_token_id] = -100

                outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
                loss = outputs.loss
                val_loss += loss.item()

        avg_val_loss = val_loss / len(val_dataloader)
        print(f"Validation loss: {avg_val_loss:.4f}")
        model.save_pretrained("/content/drive/MyDrive/fibe-1024-distilgpt2-full")
        tokenizer.save_pretrained("/content/drive/MyDrive/fibe-1024-distilgpt2-full")

    return model, tokenizer

model = AutoModelForCausalLM.from_pretrained(model_name).to("cuda")
model, tokenizer = finetune_gpt2(train_dataloader, val_dataloader, tokenizer,model, device="cuda")

# Save the fine-tuned model
model.save_pretrained("/content/drive/MyDrive/fibe-1024-distilgpt2-full")
tokenizer.save_pretrained("/content/drive/MyDrive/fibe-1024-distilgpt2-full")
from google.colab import runtime #disconnect after saving on long runs
runtime.unassign()

Epoch 1/2: 100%|██████████| 39236/39236 [6:33:32<00:00,  1.66it/s]


Average train loss: 0.2344


Validation: 100%|██████████| 17438/17438 [33:11<00:00,  8.76it/s]


Validation loss: 0.1190


Epoch 2/2: 100%|██████████| 39236/39236 [6:33:37<00:00,  1.66it/s]


Average train loss: 0.1040


Validation: 100%|██████████| 17438/17438 [32:20<00:00,  8.99it/s]


Validation loss: 0.1118


In [ ]:
import os
import json
import torch
from torch.utils.data import DataLoader, random_split
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR
from transformers import AutoModelForCausalLM, AutoTokenizer, get_linear_schedule_with_warmup
from tqdm import tqdm
# modelname = r"gpt2"
modelname = "/content/drive/MyDrive/fibe-1024-distilgpt2-full"
model = AutoModelForCausalLM.from_pretrained(modelname).to("cuda")
tokenizer = AutoTokenizer.from_pretrained(modelname,padding_side='left')
# tokenizer.pad_token = tokenizer.eos_token

In [ ]:
import numpy as np
import pandas as pd
import json
df = pd.read_csv('/content/dataset/test.csv',encoding='latin-1')

In [ ]:
def batched_inference(model, tokenizer, prompts, max_new_tokens=128, temperature=0.3, top_k=100, top_p=0.95):
    model.eval()
    dialogue_template = get_dialogue_template()
    device = "cuda"

    # Prepare batched input prompts
    input_texts = [dialogue_template.get_inference_prompt(prompt) for prompt in prompts]

    # Tokenize all prompts in a batch
    inputs = tokenizer(input_texts, return_tensors="pt", padding=True, truncation=True, max_length=1024).to(device)
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    with torch.no_grad():
      # Generate output for the whole batch
      outputs = model.generate(
          input_ids,
          attention_mask=attention_mask,
          max_new_tokens=max_new_tokens,
          temperature=temperature,
          top_k=top_k,
          top_p=top_p,
          do_sample=True,
          pad_token_id=tokenizer.pad_token_id,
      )

    # Decode each output in the batch and clean up the text
    responses = []
    for output in outputs:
        generated_text = tokenizer.decode(output, skip_special_tokens=False)
        response = generated_text.split(dialogue_template.assistant_token)[1].strip()
        response = response.replace("<|endoftext|>", "").strip()
        responses.append(response)

    return responses


In [ ]:
batch_size = 16  # Define the batch size
out_df = pd.DataFrame(columns=["target", "Index"])

# Loop through the dataframe in batches
for i in tqdm(range(0, len(df), batch_size)):
    batch = df.iloc[i:i+batch_size]  # Get a batch of rows
    lst = batch['text'].tolist()  # Extract the 'text' column (input prompts)
    prompts = []
    for txt in lst:
      cleaned = clean_text(txt)
      if len(tokenizer.tokenize(cleaned)) > 1000:
        encoded_tokens = tokenizer.encode(txt, truncation=True, max_length=1000)
        # Decode the tokens back to text (optional)
        cleaned = tokenizer.decode(encoded_tokens, skip_special_tokens=True)

      prompts.append(cleaned)

    # Get batched responses
    try:
      responses = batched_inference(model, tokenizer, prompts, top_k=1, temperature=0.01, top_p=1.0, max_new_tokens=64)
    except:
      print("Error on: ", i)
      break

    # Store the responses in out_df along with the 'Index'
    for j, response in enumerate(responses):
        out_df.loc[batch.index[j]] = [response, batch.iloc[j]['Index']]


100%|██████████| 10899/10899 [1:14:22<00:00,  2.44it/s]


In [1]:
out_df.to_csv('/content/drive/MyDrive/fibe_out_all_1024.csv')
#requires some postprocessing to remove padding tokens and adding quotes.

NameError: name 'out_df' is not defined